# S&P500



https://medium.com/mlearning-ai/predict-sp500-stock-price-with-python-machine-learning-sentiment-analysis-a296dc276353

https://www.kaggle.com/datasets/miguelaenlle/massive-stock-news-analysis-db-for-nlpbacktests

In [1]:
import pandas as pd
import numpy as np

#Title CLeaning
import string

In [2]:
#Upload Dataset
"""Columns go as follows: article title, date, stock
Timezone is UTC-4. The difference between this and raw_analyst_headlines is that this has exact dates to the minute vs. raw_analyst_ratings 
which is only the day without hour or minutes."""

df = pd.read_csv('../raw_data/analyst_ratings_processed.csv', index_col=0) 

In [3]:
df.head(10)

,title,date,stock
0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A
5.0,"CFRA Maintains Hold on Agilent Technologies, L...",2020-05-22 11:23:00-04:00,A
6.0,"UBS Maintains Neutral on Agilent Technologies,...",2020-05-22 09:36:00-04:00,A
7.0,Agilent Technologies shares are trading higher...,2020-05-22 09:07:00-04:00,A
8.0,Wells Fargo Maintains Overweight on Agilent Te...,2020-05-22 08:37:00-04:00,A
9.0,10 Biggest Price Target Changes For Friday,2020-05-22 08:06:00-04:00,A


In [4]:
df.shape

(1400469, 3)

In [5]:
df.isnull().sum().sort_values(ascending=False)/len(df)*100

stock    0.184081
date     0.092041
title    0.000000
dtype: float64

In [6]:
"""Data Cleaning
Removing Null row as low percentage"""

data = df.dropna()

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1397891 entries, 0.0 to 1413848.0
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   title   1397891 non-null  object
 1   date    1397891 non-null  object
 2   stock   1397891 non-null  object
dtypes: object(3)
memory usage: 42.7+ MB


In [8]:
""" Stripping redundant info and
    Converting to datetime format
"""


data['date'] = pd.to_datetime(data['date'].map(lambda x: str(x)[:-15]))

/tmp/ipykernel_57125/2650720732.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data['date'].map(lambda x: str(x)[:-15]))


In [9]:
data.head()

,title,date,stock
0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05,A
1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03,A
2.0,71 Biggest Movers From Friday,2020-05-26,A
3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22,A
4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22,A


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1397891 entries, 0.0 to 1413848.0
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   title   1397891 non-null  object        
 1   date    1397891 non-null  datetime64[ns]
 2   stock   1397891 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 42.7+ MB


In [11]:
""" Dropping rows with date before 2018
    as Financial statement information 
    available from 2018-2023
"""

data = data.drop(data[data['date']<pd.to_datetime("2018-01-01")].index)

In [12]:
data.tail()

,title,date,stock
1413783.0,China Zenix Auto Shares Halted News Pending,2018-06-13,ZX
1413784.0,"China Zenix Auto Q1 EPS $0.08, Made $130.123M ...",2018-05-17,ZX
1413785.0,"China Zenix Auto Reports Q4 EPS $0.03, Sales $...",2018-03-15,ZX
1413786.0,UPDATE: Not Seeing News In Chinese Momentum St...,2018-01-05,ZX
1413787.0,Chinese Nano-Cap Momentum Stocks Sharply Highe...,2018-01-05,ZX


In [13]:
data.shape

(402168, 3)

In [14]:
""" S&P 500 tickers with correlation to Financial Statement 
    Saved list to CSV
"""

tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers = tickers.Symbol.to_csv('../raw_data/tickers.csv')

In [15]:
tickers = pd.read_csv('../raw_data/tickers.csv', index_col=0)

In [16]:
""" Renaming the column to be similar with 'data' df
"""

tickers = tickers.rename(columns={'Symbol': 'stock'})

tickers

,stock
0,MMM
1,AOS
2,ABT
3,ABBV
4,ACN
...,...
498,YUM
499,ZBRA
500,ZBH
501,ZION


In [17]:
""" To filter out the S&P 500 stocks available on 'data' df 
"""

SP500 = pd.merge(data,tickers, on=['stock'], how= 'left', indicator='Exist')

In [18]:
SP500.head()

,title,date,stock,Exist
0,Stocks That Hit 52-Week Highs On Friday,2020-06-05,A,both
1,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03,A,both
2,71 Biggest Movers From Friday,2020-05-26,A,both
3,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22,A,both
4,B of A Securities Maintains Neutral on Agilent...,2020-05-22,A,both


In [19]:
""" Keeping only the filtered stocks and removing the indicator column 
"""

SP500 = SP500[SP500['Exist'] == 'both'].drop(columns='Exist')

SP500

,title,date,stock
0,Stocks That Hit 52-Week Highs On Friday,2020-06-05,A
1,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03,A
2,71 Biggest Movers From Friday,2020-05-26,A
3,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22,A
4,B of A Securities Maintains Neutral on Agilent...,2020-05-22,A
...,...,...,...
401955,Stocks That Made New 52-Wk Highs Earlier Today...,2018-02-20,ZTS
401956,Zoetis Sees FY18 Adj. EPS $2.96-$3.10 vs $2.98...,2018-02-15,ZTS
401957,Zoetis Reports Q4 Adj. EPS $0.69 vs $0.66 Est....,2018-02-15,ZTS
401958,"Earnings Scheduled For February 15, 2018",2018-02-15,ZTS


In [20]:
SP500['stock'].nunique()

392

In [21]:
#Saving the S&P Dataset for team

SP500.to_csv('../raw_data/SP500.csv')

In [22]:
#Data Cleaning
#Remove lowercase, punctuation, numbers, symbols, tokenizing

def basic_cleaning(sentence):
    sentence = sentence.lower()
    
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') 
    
    sentence = sentence.strip()
    
    return sentence

cleaned_title = [basic_cleaning(sentence) for sentence in SP500['title']]

cleaned_title

['stocks that hit 52week highs on friday',
 'stocks that hit 52week highs on wednesday',
 '71 biggest movers from friday',
 '46 stocks moving in fridays midday session',
 'b of a securities maintains neutral on agilent technologies raises price target to 88',
 'cfra maintains hold on agilent technologies lowers price target to 85',
 'ubs maintains neutral on agilent technologies raises price target to 87',
 'agilent technologies shares are trading higher after the company reported betterthanexpected q2 eps and sales results',
 'wells fargo maintains overweight on agilent technologies raises price target to 95',
 '10 biggest price target changes for friday',
 '30 stocks moving in fridays premarket session',
 'svb leerink maintains outperform on agilent technologies raises price target to 95',
 '8 stocks moving in thursdays afterhours session',
 'agilent technologies shares are trading higher after the company reported betterthanexpected q2 eps and sales results',
 'agilent technologies 

In [23]:
SP500['title'] = cleaned_title

In [24]:
SP500.to_csv('../raw_data/SP500.csv')

In [25]:
SP500.tail()

,title,date,stock
401955,stocks that made new 52wk highs earlier today ...,2018-02-20,ZTS
401956,zoetis sees fy18 adj eps 296310 vs 298 est sal...,2018-02-15,ZTS
401957,zoetis reports q4 adj eps 069 vs 066 est sales...,2018-02-15,ZTS
401958,earnings scheduled for february 15 2018,2018-02-15,ZTS
401959,zoetis 8k shows expectation for net reduction ...,2018-01-08,ZTS


In [26]:
SP500["date"].max()


Timestamp('2020-06-11 00:00:00')